# Midterm Certification Challenge: Building and Deploying a RAG Application
DUE DATE: Before 4:00 PM PT on May 2 (before next Thursday's class!)

You are to record the total time it takes you to complete

You have access to all boiler-plate code from the course, and we highly encourage you to leverage it!

**Deliverables:**

**Build 🏗️**

* Data: Meta 10-k Filings
* LLM: OpenAI GPT-3.5-turbo
* Embedding Model: text-3-embedding small
* Infrastructure: LangChain or LlamaIndex (you choose)
* Vector Store: Qdrant
* Deployment: Chainlit, Hugging Face
**Ship 🚢**

* Evaluate your answers to the following questions
"What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
"Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"
* Record <10 min loom video walkthrough
$$ Extra Credit: Baseline retrieval performance w/ RAGAS, change something about your RAG system to improve it, then show the improvement quantitatively!

**Share 🚀**
* Share lessons not yet learned in #aie2-general

## Install Dependencies

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
!pip install llama-parse llama_index -qU

In [6]:
!pip install -qU langchain langchain-core langchain-community langchain-openai unstructured

In [7]:
!pip install -qU qdrant-client

## Set Environment Variables

In [13]:
import os
from getpass import getpass

# set openai key
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

OpenAI API Key:··········


In [9]:
# set llama cloud key
os.environ["LLAMA_CLOUD_API_KEY"] = getpass("Llama Cloud API Key:")

Llama Cloud API Key:··········


## Download the Data

In [10]:
# download the data
!mkdir 'data'
!wget 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf' -O 'data/Meta_10k.pdf'

--2024-05-02 18:27:08--  https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf
Resolving d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)... 18.154.131.210, 18.154.131.173, 18.154.131.90, ...
Connecting to d18rn0p25nwr6d.cloudfront.net (d18rn0p25nwr6d.cloudfront.net)|18.154.131.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2481466 (2.4M) [application/pdf]
Saving to: ‘data/Meta_10k.pdf’

data/Meta_10k.pdf   100%[===================>]   2.37M  --.-KB/s    in 0.05s   

2024-05-02 18:27:08 (47.1 MB/s) - ‘data/Meta_10k.pdf’ saved [2481466/2481466]



## RAG with Llama Parse + LangChain RecursiveCharacterTextSplitter

First, we'll parse the document using Llama Parse. Then we'll save the llama_parse markdown document so we can use it later.

In [71]:
# import dependencies
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

parsing_instruction = """The provided document is an annual report filed by Meta Platforms, Inc. with the Securities and Exchange Commission (SEC).
This form provides detailed financial information about the company's performance for a specific year.
It includes financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables and some signature pages.

Replace the signatures with tables containing the headers for each element.
"""

# setup parser
parser = LlamaParse(
    result_type="markdown",
    parsing_instruction=parsing_instruction
)

# load and parse the documet
file_extractor = {".pdf": parser}
llama_parse_documents = SimpleDirectoryReader(
    input_files=['data/Meta_10k.pdf'],
    file_extractor=file_extractor
).load_data()

# save markdown file
data_file = "./data/output.md"
with open(data_file, "a") as f:
    for doc in llama_parse_documents:
        f.write(doc.text + '\n')

Started parsing the file under job_id f27fcb88-f758-4d41-b1ab-d38b8daf6754
....

Now we'll setup the langchain RAG with Qdrant

In [83]:
# import dependencies
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai.embeddings import OpenAIEmbeddings

# load the document
loader = DirectoryLoader(path='data/', glob="**/*.md", show_progress=True)
documents = loader.load()

# split the document into chunks

# split markdown headers
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

md_text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers = False
)

md_splits = md_text_splitter.split_text(documents[0].page_content)

# recursive character text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500, chunk_overlap=100)
docs = text_splitter.split_documents(md_splits)

# instantiate embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# create the vectorstore
qdrant_vector_store = Qdrant.from_documents(
    documents=docs,
    embedding=embeddings,
    location=":memory:",
    collection_name="meta_10k"
)

# setup our retriever
qdrant_retriever = qdrant_vector_store.as_retriever()

100%|██████████| 1/1 [00:16<00:00, 16.71s/it]


Next, we'll setup the RAG Prompt.

In [84]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

The provided context is an annual report filed by Meta Platforms, Inc. with the Securities and Exchange Commission (SEC).
This form provides detailed financial information about the company's performance for a specific year.
It includes financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables and some signature pages. All members of the board need to sign the document.

Answer the query above only using the context provided. If you don't know the answer, simply say 'I don't know'.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

Finally, we create our chain...

In [85]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo")

rag_chain = (
    {"question": itemgetter("question"), "context": itemgetter("question") | qdrant_retriever}
    | RunnablePassthrough().assign(context=itemgetter("context"))
    | {"response":rag_prompt | chat_model | StrOutputParser(), "context": itemgetter("context")}
)

## Query the Meta 10-K Form

Great, time to query the Form!

In [87]:
# query the rag_chain
query1 = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
response = rag_chain.invoke({"question": query1})
print(response['response'])

The total value of 'Cash and cash equivalents' as of December 31, 2023, was $41,862 million.


In [88]:
for context in response['context']:
    print('======== CONTEXT ========')
    print(context.page_content)

======== CONTEXT ========
Fair Value Measurement at Reporting Date Using
|Description|December 31, 2023|Quoted Prices in Active Markets for Identical Assets (Level 1)|Significant Observable Inputs (Level 2)|Significant Unobservable Inputs (Level 3)|
|---|---|---|---|---|
|Cash|$6,265| | | |
|Cash equivalents: Money market funds|$32,910|$32,910| | |
|Cash equivalents: U.S. government and agency securities|$2,206|$2,206| | |
|Cash equivalents: Time deposits|$261| |$261| |
|Cash equivalents: Corporate debt securities|$220| |$220| |
|Total cash and cash equivalents|$41,862|$35,116|$481| |
|Marketable securities: U.S. government securities|$8,439|$8,439| | |
|Marketable securities: U.S. government agency securities|$3,498|$3,498| | |
|Marketable securities: Corporate debt securities|$11,604| |$11,604| |
|Total marketable securities|$23,541|$11,937|$11,604| |
|Restricted cash equivalents|$857|$857| | |
|Other assets|$101| | |$101|
|Total|$66,361|$47,910|$12,085|$101|

107

Meta Platforms, In

In [89]:
# query the rag_chain
query2 = "Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"
response = rag_chain.invoke({"question": query2})
print(response['response'])

The Directors of Meta Platforms, Inc. listed in the document are:

- Mark Zuckerberg
- Susan Li
- Aaron Anderson
- Peggy Alford
- Marc L. Andreessen
- Andrew W. Houston
- Nancy Killefer
- Robert M. Kimmitt
- Sheryl K. Sandberg
- Tracey T. Travis
- Tony Xu


In [90]:
for context in response['context']:
    print('======== CONTEXT ========')
    print(context.page_content)

======== CONTEXT ========
10

Meta Platforms, Inc. - Annual Report

Table of Contents

Table of contents content goes here...

Signatory Title Date Mark Zuckerberg Chief Executive Officer March 1, 2024 Sheryl Sandberg Chief Operating Officer March 1, 2024 David Wehner Chief Financial Officer March 1, 2024 --- # Meta Platforms, Inc. Annual Report

Meta Platforms, Inc. Annual Report

Signatures

Name Title Date [Signature Name 1] [Title 1] [Date 1] [Signature Name 2] [Title 2] [Date 2] [Signature Name 3] [Title 3] [Date 3] --- # Meta Platforms, Inc. Annual Report

Table of Contents

Compensation, Benefits, Health, and Well-being

We offer competitive compensation to attract and retain the best people, and we help care for our people so they can focus on our mission. Our employees' total compensation package includes market-competitive salary, bonuses or sales incentives, and equity. We generally offer full-time employees equity at the time of hire and through annual equity grants because

In [91]:
# query the rag_chain
response = rag_chain.invoke({"question": "What's the par value of Meta's Class A common stock?"})
print(response['response'])

The par value of Meta's Class A common stock is $0.000006 per share.


In [95]:
# query the rag_chain
response = rag_chain.invoke({"question": "What is Meta's dividend policy?"})
print(response['response'])

Meta's dividend policy states that prior to 2024, the company had never declared or paid any cash dividend on their common stock. However, on February 1, 2024, they announced the initiation of their first ever cash dividend program. This program includes a cash dividend of $0.50 per share of Class A common stock and Class B common stock, equivalent to $2.00 per share on an annual basis. The first cash dividend was scheduled to be paid on March 26, 2024 to all holders of record of common stock at the close of business on February 22, 2024. The payment of future cash dividends is subject to future declaration by their board of directors, based on various factors including capital availability, market conditions, laws, and the best interests of stockholders.


In [96]:
# query the rag_chain
response = rag_chain.invoke({"question": "What is Meta's current net worth?"})
print(response['response'])

Meta's current net worth, based on the information provided in the annual report, is $229,623 million as of December 31, 2023.
